#Project Euler problem 286 (Scoring probabilities)

## Problem statement

Barbara is a mathematician and a basketball player. She has found that the probability of scoring a point when shooting from a distance x is exactly (1 - x/q), where q is a real constant greater than 50.

During each practice run, she takes shots from distances x = 1, x = 2, ..., x = 50 and, according to her records, she has precisely a 2 % chance to score a total of exactly 20 points.

Find q and give your answer rounded to 10 decimal places.


##Mathematical interlude

First of all I'd like to find a formula for $P(\alpha | x )$, where $\alpha$ is the number of points scored and $x$ is the number of shots taken (aka the number of "available" distances).

Let's start by looking into $P(\alpha=1 | x=50)$: 

\begin{equation}
\begin{split}
P(1) &= (1-\frac{1}{q})(\frac{2}{q})(...)(\frac{50}{q}) \\ 
       & + (\frac{1}{q})(1-\frac{2}{q})(...)(\frac{50}{q}) \\ 
       & + \dots \\
       & +(\frac{1}{q})(\frac{2}{q})(...)(1-\frac{50}{q})
\end{split}
\end{equation}



This seems to be amenable to be written as something like

$$
P(1) = \frac{1}{q^{50}} \sum_{k=1}^{50}\left[(\prod_{j=1}^{k-1} j)(q-k)(\prod_{i=k+1}^{50}i)\right]
$$

Or less sadly:

$$
P(1) = \frac{1}{q^{50}} \prod_{j=1}^{50}j\sum_{k=1}^{50}\frac{q-k}{k}
$$

Noting that $\prod_{j=1}^{50}j = 50!$  and $\frac{q-k}{k} = \frac{q}{k}-1$, and generalizing for arbitrary number of distances $n$ (rather than 50), $P(1)$ can be finally written as

$$
P(\alpha=1 | x=n) = \frac{n!}{q^{n}}\sum_{k=1}^{n}(\frac{q}{k}-1)
$$

Let's see what happens when we're looking for the probability of scoring (exactly) two points. This is the probability of making the first and second shots and missing all others, or making the first and third and missing all others, etc., or making the second and third and missing all others, etc., or making the 49th and 50th after having missed all others. In math:

\begin{equation}
\begin{split}
P(2) &= (1-\frac{1}{q})(1-\frac{2}{q})(...)(\frac{50}{q}) \\
       & + (1-\frac{1}{q})(\frac{2}{q})(1-\frac{3}{q})(...)(\frac{50}{q}) \\ 
       & + \dots \\
       & +(1-\frac{1}{q})(\frac{2}{q})(...)(1-\frac{50}{q}) \\
       & + (\frac{1}{q})(1-\frac{2}{q})(1-\frac{3}{q})(...)(\frac{50}{q}) \\ 
       & + \dots \\
       & +(\frac{1}{q})(1-\frac{2}{q})(...)(1-\frac{50}{q}) \\
       & + \dots \\
       & + (\frac{1}{q})(\frac{2}{q})(...)(1-\frac{49}{q})(1-\frac{50}{q})
\end{split}
\end{equation}

Which translates to something along the lines of

\begin{equation}
\begin{split}
P(2) &= \frac{50!}{q^{50}} \left[ (q-1)\sum_{k=2}^{50}\frac{q-k}{1\times k} + (q-2)\sum_{k=3}^{50}\frac{q-k}{2\times k} + \dots + (q-49)\sum_{k=50}^{50}\frac{q-k}{49\times k}\right] \\
    &= \frac{50!}{q^{50}} \left( \sum_{j=1}^{49}\frac{q-j}{j} \times \left( \sum_{k=j+1}^{50}\frac{q-k}{k} \right)\right)
\end{split}
\end{equation}

By using magical abilities this can be generalized for arbitrary $\alpha$ and $n$ (with $n \geq \alpha$) as follows:

$$
P(\alpha | n) = \frac{n!}{q^n} \sum_{k_1=1}^{n-(\alpha-1)}\frac{q-k_1}{k_1} \sum_{k_2=k_1+1}^{n-(\alpha-2)}\frac{q-k_2}{k_2}\sum_{k_3=k_2+1}^{n-(\alpha-3)}\frac{q-k_3}{k_3} \dots \sum_{k_\alpha=k_{\alpha-1}+1}^{n}\frac{q-k_\alpha}{k_\alpha}
$$

Some comments:
- there are exactly $\alpha$ summation terms
- there is probably a way of using combinatory to simplify this

## Main event: finding $q$

Let's start with a simple example. Let's say Barbara takes 3 shots and the probability of scoring exactly two points is $20\%$.

- $\alpha = 2$
- $n = 3$
- $P(2|3) = .2$


From the interlude we have

$$
P = \frac{3!}{q^3} \sum_{k_1=1}^{2} \frac{q-k_1}{k_1} \sum_{k_2=k_1+1}^{3} \frac{q-k_2}{k_2} = 0.2
$$

and after some algebra we arrive at a nice expression that we can solve for $q$:

$$
-0.2q^3 + 3q^2 -12q + 11 = 0
$$

The bad news is that this works for $n=3$ and $n=4$, maybe $n=5$, but not further on, and we need $n=50$.

So we'll need an alternative strategy.

### Trying Numpy and Scipy
(reference: http://stackoverflow.com/questions/10457240/solving-polynomial-equations-in-python)

In [22]:
import numpy as np
q = [-.2, 3, -12,11]
np.roots(q)


array([ 9.02997548,  4.66414098,  1.30588355])

So apparently Numpy is one possible way to go. All we have to do is compute the coeffiecients for the polynomial.
Can we use Sympy to find those?


In [133]:
from sympy import symbols, summation, expand
from sympy.solvers import solve

# define problem variables
# alpha: how many points
# n: how many shots

alpha = 2
n = 3

q,k = symbols('q k')

expr = (q-k)/k


def nestsums(expr, suminit, sumend, alpha, n):
    
    if suminit == alpha: # ultimo termo
        return sumq(expr, suminit, sumend)
    else:
        return sumq(expr, suminit, sumend) * nestsums(expr, suminit+1, sumend+1, alpha, n) #- (sumq(expr, 2,2))**2

# This won't work because it keeps quadratic terms (e.g. (q-2)*(q-2))
# We should find a way of implementing the theoretical formula

def sumq(expr, k0, n):
    return summation (expr, (k,k0,n))


In [134]:
# tests
def test_sumq():
    from sympy import symbols
    q,k = symbols('q k')
    exp = (q-k)/k
    
    t1 = sumq(exp, 1, 3)
    assert t1 == (11*q-18)/6
    t2 = sumq(exp,1,2)
    assert t2 == (3*q/2 - 2)
    t3 = sumq(exp,2,3)
    assert t3 == (5*q/6-2)
    t4 = sumq(exp,8,8)
    assert t4 == (q/8-1)
    
def test_nestsums():
    from sympy import symbols, expand
    q,k = symbols('q k')
    exp = (q-k)/k
    t1 = nestsums(exp, 1, 2, 2, 3)
    t1 = expand(t1)
    try:
        assert t1 == (q**2 -11*q/3 +3)
    except AssertionError:
        print t1
        raise AssertionError
                
test_sumq()
test_nestsums()

5*q**2/4 - 14*q/3 + 4


AssertionError: 

### State of the art as of 19-8-2015
I'll leave this incomplete as I can't find a way to implement the formula to obtain the coefficients for the $q^k$ terms. A solution doesn't seem to be too far, but no time to keep at it now. Hope to come back to this problem soon.